# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-5-nano'
openai = OpenAI()

API key looks good so far


In [3]:
links = fetch_website_links("https://edwarddonner.com")
links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 '

## First step: Have GPT-5-nano figure out which links are relevant

### Use a call to gpt-5-nano to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [4]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [5]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [6]:
print(get_links_user_prompt("https://edwarddonner.com"))


Here is the list of links on the website https://edwarddonner.com -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/
https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/
https://edwar

In [7]:
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links
    

In [8]:
select_relevant_links("https://edwarddonner.com")

{'links': [{'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'home page', 'url': 'https://edwarddonner.com/'}]}

In [9]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [10]:
select_relevant_links("https://edwarddonner.com")

Selecting relevant links for https://edwarddonner.com by calling gpt-5-nano
Found 4 relevant links


{'links': [{'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'company page',
   'url': 'https://nebula.io/?utm_source=ed&utm_medium=referral'},
  {'type': 'homepage', 'url': 'https://edwarddonner.com/'},
  {'type': 'professional profile',
   'url': 'https://www.linkedin.com/in/eddonner/'}]}

In [11]:
select_relevant_links("https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 10 relevant links


{'links': [{'type': 'about page', 'url': 'https://huggingface.co/brand'},
  {'type': 'company page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'product page', 'url': 'https://endpoints.huggingface.co'},
  {'type': 'GitHub', 'url': 'https://github.com/huggingface'},
  {'type': 'Twitter', 'url': 'https://twitter.com/huggingface'},
  {'type': 'LinkedIn', 'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'community forum', 'url': 'https://discuss.huggingface.co'},
  {'type': 'Discord', 'url': 'https://huggingface.co/join/discord'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT-5-nano

In [12]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [13]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 16 relevant links
## Landing Page:

Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
deepseek-ai/DeepSeek-OCR
Updated
2 days ago
•
731k
•
1.93k
PaddlePaddle/PaddleOCR-VL
Updated
3 days ago
•
15.6k
•
1.09k
tencent/HunyuanWorld-Mirror
Updated
2 days ago
•
8.31k
•
348
krea/krea-realtime-video
Updated
6 days ago
•
1.15k
•
204
Qwen/Qwen3-VL-8B-Instruct
Updated
11 days ago
•
290k
•
333
Browse 1M+ models
Spaces
Running
on
Zero
264
264
DeepSeek OCR Demo
🆘
An interactive demo for the DeepSeek-OCR model.
Running
15.4k
15.4k
DeepSite v3
🐳
Generate any application by Vibe Coding
Running
529
529
veo3.1-fast
🐨
Generate videos from

In [22]:
# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""


In [15]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [16]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 11 relevant links


'\nYou are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nHugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\ndeepseek-ai/DeepSeek-OCR\nUpdated\n2 days ago\n•\n731k\n•\n1.93k\nPaddlePaddle/PaddleOCR-VL\nUpdated\n3 days ago\n•\n15.6k\n•\n1.09k\ntencent/HunyuanWorld-Mirror\nUpdated\n2 days ago\n•\n8.31k\n•\n348\nkrea/krea-realtime-video\nUpdated\n6 days ago\n•\n1.15k\n•\n204\nQwen/Qwen3-VL-8B-Instruct\nUpdated\n11 days ago\n•\n290k\n•\n333\nBrowse 1M+ models\nSpaces\nRunning\non\nZero\n264\n264\nDeep

In [17]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [18]:
create_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 4 relevant links


# Hugging Face Brochure

---

## About Hugging Face

Hugging Face is a pioneering AI community and collaboration platform focused on building the future of machine learning and artificial intelligence. It serves as the central hub where machine learning engineers, scientists, and AI enthusiasts connect to create, share, discover, and experiment with millions of open-source models, datasets, and applications across various modalities such as text, image, video, audio, and 3D.

The company is recognized for empowering the next generation of AI developers with an open and ethical approach to AI development, driving forward the AI revolution with a fast-growing community and cutting-edge science.

---

## What Hugging Face Offers

### Platform Features
- **Hugging Face Hub:** Host and collaborate on unlimited public ML models, datasets, and applications.
- **Extensive Library:** Access over 1 million pre-trained machine learning models and browse through 250,000+ datasets.
- **Spaces:** Run and demo interactive AI applications directly on the platform.
- **Multimodal Support:** Work seamlessly with text, image, video, audio, and 3D data formats.
- **Open Source Stack:** Accelerate your machine learning development with a robust open-source software stack.

### Enterprise Solutions
- **Team & Enterprise Plans:** Secure and scalable solutions starting at $20 user/month with options to customize for enterprise needs.
- **Enterprise-grade Security:** Features include Single Sign-On (SSO), audit logs, granular access controls, token management, and private storage.
- **Enhanced Collaboration:** Tools for managing data locations, resource groups, private dataset viewing, and analytics dashboards.
- **Advanced Compute Options:** Higher scalability with features like ZeroGPU quota boosts.
- **Dedicated Support:** Receive professional assistance tailored to organizational requirements.

---

## Company Culture

Hugging Face embraces an open, collaborative, and inclusive culture centered on community building and ethical AI innovation. The company thrives on the contributions of its users — from hobbyists to top industry scientists — fostering a vibrant ecosystem where everyone can learn, share, and progress together.

Their brand emphasizes openness and transparency, encouraging users to build their individual portfolios and machine learning profiles by sharing work on this collaborative platform.

---

## Customers and Community

Hugging Face’s customer base includes:
- AI researchers and engineers
- Data scientists and machine learning practitioners
- Enterprises seeking scalable AI deployment and collaboration tools
- Open-source contributors and developers focusing on AI tools and applications

Its community spans millions worldwide, contributing to a vast and continuously evolving repository of ML models and datasets. The platform's extensive resources support innovation across various industries and research domains.

---

## Careers at Hugging Face

Hugging Face invites talented individuals passionate about AI and open-source technology to join their team. The company values diversity, creativity, and technical excellence, offering opportunities in engineering, research, product development, and community engagement.

Working at Hugging Face means contributing to the forefront of AI technology, collaborating with a skilled international team, and helping to shape an open and ethical future for machine learning.

For current openings and application details, prospective candidates are encouraged to visit the company’s **Jobs** page.

---

## Connect with Hugging Face

- **Website:** https://huggingface.co  
- **Community & Support:** Active forum, Discord, GitHub  
- **Social Media:** Twitter, LinkedIn  
- **Documentation & Tutorials:** Comprehensive guides and blog posts for all users  

---

## Brand Highlights

- Signature Colors: Yellow (#FFD21E), Orange (#FF9D00), Gray (#6B7280)  
- Recognized as the home of collaborative open-source AI  
- Trusted for both individual creators and enterprise teams  

---

**Join Hugging Face today and be part of the AI community building tomorrow’s technology—open, collaborative, and ethical.**

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [19]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [21]:
stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 9 relevant links


# Hugging Face Brochure

---

## About Hugging Face

Hugging Face is the leading AI community and collaboration platform shaping the future of machine learning. Serving as a central hub, Hugging Face empowers AI developers, researchers, and enthusiasts to create, discover, and collaborate on models, datasets, and applications. Their platform hosts over 1 million machine learning models and hundreds of thousands of datasets spanning text, image, video, audio, and 3D modalities.

The company’s mission is to build an open and ethical AI future by fostering a fast-growing community where machine learning engineers, scientists, and end users can share their work, learn from each other, and accelerate innovation.

---

## What Hugging Face Offers

### Collaboration Platform
- Host, share, and collaborate on unlimited **public models**, **datasets**, and **applications**.
- Build and showcase your ML portfolio by sharing your innovations with the world.

### Extensive Resources
- Access to **1M+ models** covering multiple modalities.
- Explore **250k+ datasets** curated for diverse AI projects.
- Discover and run **400k+ AI applications** in diverse areas such as OCR, video generation, natural language processing, and more.

### Community & Open Source
- A thriving global community dedicated to open source AI.
- Integration with the HF Open Source stack accelerates model development and deployment.

### Enterprise Solutions
- Dedicated **Enterprise** platform with advanced security, access control, and priority support.
- **Compute solutions** for scalable, paid resources to accelerate AI projects.
- Ideal for teams seeking collaboration with enterprise-grade features.

---

## Company Culture

- **Community-Centric:** At its core, Hugging Face is a community-driven company that values openness, collaboration, and knowledge sharing.
- **Inclusive & Ethical AI:** Emphasizes building trustworthy AI technologies that benefit everyone.
- **Innovative & Fast-Paced:** Constantly evolving with cutting-edge advancements in AI and machine learning.
- **Supportive Environment:** Offers tools and resources to support both beginners and expert practitioners in advancing their skills and projects.

---

## Customers & Use Cases

Hugging Face serves a diverse user base, ranging from independent researchers and developers to large enterprises and academic institutions. Their platform is widely used for:

- Natural Language Processing tasks such as chatbots, text generation, and sentiment analysis.
- Computer Vision applications including OCR, image and video generation.
- Speech and audio processing workflows.
- Developing and deploying multimodal AI applications combining text, image, and video inputs.

---

## Careers & Opportunities

Hugging Face is expanding rapidly and offers exciting career opportunities across engineering, research, product management, and customer success roles. The company seeks motivated individuals who are passionate about AI, open source, and community-building.

**Why Join Hugging Face?**
- Collaborate with an international, diverse team of AI experts.
- Work on challenging, impactful problems shaping the future of AI.
- Be part of a mission-driven organization committed to an open and ethical AI ecosystem.
- Access to resources and a creative environment that fosters learning and innovation.

Visit [Hugging Face Careers](https://huggingface.co/jobs) to explore open positions and apply.

---

## Get Started

Join the world’s largest AI community today:

- Explore and contribute to millions of models and datasets.
- Launch AI applications instantly through Hugging Face Spaces.
- Build your machine learning portfolio and collaborate with innovators worldwide.

**Sign up now at [huggingface.co](https://huggingface.co) and start building the future of AI together.**

---

*Hugging Face – The AI community building the future.*

In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano


<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>